## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## Library Imports

In [ ]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib inline

# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

from scipy.ndimage.interpolation import shift

source=[]

## Camera Calibration Function

In [ ]:
def CalibrateCamera():
    """
    This function calibrates the camera utilizing provided chessboard images
    """
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((6*9,3), np.float32)
    objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.

    # Make a list of calibration images
    images = glob.glob('./camera_cal/calibration*.jpg')
    
    # Step through the list and search for chessboard corners
    for fname in images:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)

            # Draw and display the corners
            img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
            cv2.imshow('img',img)
            cv2.waitKey(500)
            
    # Camera calibration, given object points, image points, and the shape of the grayscale image
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    
    cv2.destroyAllWindows()
    
    return mtx,dist,images

## Undistort Image Function

In [ ]:
def UndistortImage(img, camMatrix, distCoeffs):
    """
    This function undistorts an image using the camera matrix and the distortion coefficients
    """    
    dest = cv2.undistort(img, camMatrix, distCoeffs, None, camMatrix)
    return dest

## Color Conversion & Thresholding Function (HLS)

In [ ]:
def ColorThresholdingHLS(image,thresh):
    """
    This function converts from RGB to the HLS color space and returns a thresholded binary image
    """
    hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    H = hls[:,:,0]
    L = hls[:,:,1]
    S = hls[:,:,2]

    binary = np.zeros_like(S)
    binary[(S > thresh[0]) & (S <= thresh[1])] = 1
    return binary

## Color Conversion & Thresholding Function (LUV)

In [ ]:
def ColorThresholdingLUV(image,thresh):
    """
    This function converts from RGB to the LUV color space and returns a thresholded binary image
    """
    luv = cv2.cvtColor(image, cv2.COLOR_RGB2Luv)
    L = luv[:,:,0]
    U = luv[:,:,1]
    V = luv[:,:,2]

    binary = np.zeros_like(L)
    binary[(L >= thresh[0]) & (L <= thresh[1])] = 1
    return binary

## Color Conversion & Thresholding Function (LAB)

In [ ]:
def ColorThresholdingLAB(image,thresh):
    """
    This function converts from RGB to the LAB color space and returns a thresholded binary image
    """
    lab = cv2.cvtColor(image, cv2.COLOR_RGB2Lab)
    L = lab[:,:,0]
    A = lab[:,:,1]
    B = lab[:,:,2]

    binary = np.zeros_like(B)
    binary[(B > thresh[0]) & (B <= thresh[1])] = 1
    return binary

## Gradient Magnitude Thresholding Function 

In [ ]:
def GradientMagThreshold(image, sobelKernel=3, magThresh=(0, 255)):
    """
    This function computes the gradient magnitude and returns a thresholded binary image
    """
    # Apply the following steps to image
    # (1) Convert to grayscale
    gray = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
    
    # (2) Take the gradient in x and y separately
    sobelx = cv2.Sobel(gray,cv2.CV_64F,1,0,sobelKernel)
    sobely = cv2.Sobel(gray,cv2.CV_64F,0,1,sobelKernel)
    
    # (3) Calculate the magnitude 
    absSobelx = np.absolute(sobelx)
    absSobely = np.absolute(sobely)
    absSobel  = np.sqrt(absSobelx**2+absSobely**2)
    absSobel  = absSobelx
    
    # 4) Scale to 8-bit (0 - 255) and convert to type = np.uint8
    scaledSobel = np.uint8(255*absSobel/np.max(absSobel))
    
    # 5) Create a binary mask where magnitude thresholds are met
    binary = np.zeros_like(scaledSobel)
    binary[(scaledSobel >= magThresh[0]) & (scaledSobel <= magThresh[1])] = 1
    
    # 6) Return this mask as the binary output image
    return binary

## Gradient Direction Thresholding Function

In [ ]:
def GradientDirThreshold(image, sobelKernel=3, dirThresh=(0, np.pi/2)):
    """
    This function computes the direction of the gradient and returns a thresholded binary image
    """    
    # Apply the following steps to image
    # (1) Convert to grayscale
    gray = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
    
    # (2) Take the gradient in x and y separately
    sobelx = cv2.Sobel(gray,cv2.CV_64F,1,0,ksize = sobelKernel)
    sobely = cv2.Sobel(gray,cv2.CV_64F,0,1,ksize = sobelKernel)
    
    # (3) Take the absolute value of the x and y gradients
    absSobelx = np.absolute(sobelx)
    absSobely = np.absolute(sobely)
    
    # (4) Use np.arctan2(abs_sobely, abs_sobelx) to calculate the direction of the gradient 
    dirGrad = np.arctan2(absSobely, absSobelx)
    
    # (5) Create a binary mask where direction thresholds are met
    binary = np.zeros_like(dirGrad)
    binary[(dirGrad >= dirThresh[0]) & (dirGrad <= dirThresh[1])] = 1
    
    # (6) Return this mask as your binary_output image
    return binary

## Mask Implementation Function

In [ ]:
def ApplyMasking(image):
    """
    This function removes the upper half of the image and returns the bottom half
    """    
    # Define the verticies for the masked region
    imshape  = image.shape
    
    vertices = np.array([[(150,imshape[0]),
                          (imshape[1]/2.0-55, imshape[0]/2.0+75), 
                          (imshape[1]/2.0+55, imshape[0]/2.0+75), 
                          (imshape[1]-10,imshape[0])]], dtype=np.int32)

    # Initialize the mask
    mask = np.zeros_like(image)   

    # Define a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(image.shape) > 2:
        channel_count = image.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    # Fill pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    # Return the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(image, mask)
    return masked_image

## Find Lane Line Start Function

In [ ]:
def FindLaneLineStart(binary_warped):
    """
    This function detects the beginning of the lane lines at the bottom of the image
    """
    # Run this function on the warped binary image
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[np.int32(binary_warped.shape[0]/2):,:], axis=0)
    
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftxBase = np.argmax(histogram[:midpoint])
    rightxBase = np.argmax(histogram[midpoint:]) + midpoint

    return leftxBase,rightxBase

## Track Sliding Windows Function

In [ ]:
def TrackSlidingWindows(nwindows,binary_warped,leftx_base,rightx_base):
    """
    This function tracks the lane lines from the bottom to the top of the image using sliding windows
    """
    # If we need to do a blind search, do the following
    if True:
        # Set height of windows
        window_height = np.int(binary_warped.shape[0]/nwindows)
        
        # Identify the x and y positions of all nonzero pixels in the image
        nonzero = binary_warped.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        
        # Current positions to be updated for each window
        leftx_current = leftx_base
        rightx_current = rightx_base
        
        # Set the width of the windows +/- margin
        margin = 100
        
        # Set minimum number of pixels found to recenter window
        minpix = 50
        
        # Create empty lists to receive left and right lane pixel indices
        left_lane_inds = []
        right_lane_inds = []
        
        # Output image for visualization
        out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255

        # Step through the windows one by one
        for window in range(nwindows):
            # Identify window boundaries in x and y (and right and left)
            win_y_low = binary_warped.shape[0] - (window+1)*window_height
            win_y_high = binary_warped.shape[0] - window*window_height
            win_xleft_low = leftx_current - margin
            win_xleft_high = leftx_current + margin
            win_xright_low = rightx_current - margin
            win_xright_high = rightx_current + margin
            
            # Draw the windows on the visualization image
            cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
            cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 

            # Identify the nonzero pixels in x and y within the window
            good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
            good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
            
            # Append these indices to the lists
            left_lane_inds.append(good_left_inds)
            right_lane_inds.append(good_right_inds)
            
            # If you found > minpix pixels, recenter next window on their mean position
            if len(good_left_inds) > minpix:
                leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
            if len(good_right_inds) > minpix:        
                rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

        # Concatenate the arrays of indices
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    # Else, if no blind search is necessary
    else:
        # from the next frame of video (also called "binary_warped")
        # It's now much easier to find line pixels!
        nonzero = binary_warped.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        margin = 100
        left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] + margin))) 
        right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] + margin)))  

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    return left_lane_inds,right_lane_inds,nonzerox,nonzeroy,leftx,lefty,rightx,righty,out_img

## Polynomial Fit To Sliding Windows Function

In [ ]:
def FitPolynomialToSlidingWindows(binary_warped,leftx,lefty,rightx,righty):
    """
    This function fits a polynomial to the lane lines
    """
    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    return left_fit,right_fit,left_fitx,right_fitx,ploty

## Lane Line Visualization Function

In [ ]:
def VisualizeLaneLines(binary_warped,left_lane_inds,right_lane_inds,nonzerox,nonzeroy,left_fit,right_fit,margin):
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(out_img)
    # Color in left and right line pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int32([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int32([right_line_pts]), (0,255, 0))
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    plt.imshow(result)
    plt.plot(left_fitx, ploty, color='yellow')
    plt.plot(right_fitx, ploty, color='yellow')
    plt.xlim(0, 1280)
    plt.ylim(720, 0)
    plt.show()
    
    return result

## Draw Full Lane Extent Function

In [ ]:
def DrawFullLane(undistorted,binary_warped,Minv,left_fit,right_fit):
    """
    This function draws the full extent of the lane
    """
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    
    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int32([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (color_warp.shape[1], color_warp.shape[0]))
    
    # Combine the result with the original image
    result = cv2.addWeighted(undistorted, 1, newwarp, 0.3, 0)
    
    return result

## Compute Lane Curvature Function

In [ ]:
def ComputeLaneCurvature(binary_warped,left_fit,right_fit):
    """
    This function computes the lane curvature
    """
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    leftx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    rightx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    # Fit a second order polynomial to pixel positions in each fake lane line
    left_fit = np.polyfit(ploty, leftx, 2)
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fit = np.polyfit(ploty, rightx, 2)
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    # Define y-value where we want radius of curvature
    # I'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    left_curverad = ((1 + (2*left_fit[0]*y_eval + left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])
    right_curverad = ((1 + (2*right_fit[0]*y_eval + right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])

    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 20/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/800 # meters per pixel in x dimension

    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(ploty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, rightx*xm_per_pix, 2)
    
    # Calculate the new radii of curvature
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    
    return left_curverad,right_curverad,leftx,rightx,ploty,left_fitx,right_fitx

## Compute Offset From Center

In [ ]:
def ComputeOffsetFromCenter(src,image):
    # Get the x-coordinates of the lane-line start points
    leftx  = src[1][0]
    rightx = src[2][0]
    
    xm_per_pix = 3.7/800 # meters per pixel in x dimension

    # Compute distance of the above points from the edges of the video frame
    d_l = leftx
    d_r = image.shape[1]-rightx
        
    if d_l > d_r:
        return (d_l-(d_l+d_r)/2)*xm_per_pix
    elif d_l < d_r:
        return (d_r-(d_l+d_r)/2)*xm_per_pix
    else:
        return 0

## Determine Source Points For Perspective Transform

In [ ]:
def GetPerspectiveSourcePoints(image):
    """
    This function finds four corners in the original image to use as source points in the perspective transform
    """ 
    # start scanning rows from the bottom of the image and identify the
    # pixel coordinates for a pair of lane line points closest to the bottom
    src = []
    srcTop = {}
    srcBottom = {}
    margin = 10
    left_done  = False
    right_done = False
    row_midpoint = int(image.shape[1]/2)
    for i in np.arange(image.shape[0]-55,np.int32(image.shape[0]/2+75),-1):
        row = image[i,:]
        left_row = row[:row_midpoint-margin]
        right_row = row[row_midpoint+margin:]
        left_ind = np.where(left_row == 1)
        right_ind = np.where(right_row == 1)
        if len(left_ind[0] != 0) and left_done == False:
            min_left_ind  = left_ind[0][0]
            max_left_ind  = left_ind[0][-1]
            med_left_ind  = left_ind[0][np.int32(len(left_ind[0])/2)]
            mean_left_ind = np.int32((min_left_ind+max_left_ind)/2)
            srcBottom[0] = [max_left_ind,i]
            #srcBottom[0] = [mean_left_ind,i]
            left_done = True
        if len(right_ind[0] != 0) and right_done == False:           
            min_right_ind = right_ind[0][0]
            max_right_ind = right_ind[0][-1]
            med_right_ind = right_ind[0][np.int32(len(right_ind[0])/2)]
            mean_right_ind = np.int32((min_right_ind+max_right_ind)/2)
            srcBottom[1] = [row_midpoint+margin+min_right_ind,i]            
            #srcBottom[1] = [row_midpoint+margin+mean_right_ind,i]            
            right_done = True
        if right_done == True and left_done == True:
            break
        
    # start scanning rows from the top of the image and identify the
    # pixel coordinates for a pair of lane line points closest to the top
    row_midpoint = int(image.shape[1]/2)
    left_done  = False
    right_done = False
    for i in np.arange(400,image.shape[0]):
        row = image[i,:]
        left_row = row[:row_midpoint-margin]
        right_row = row[row_midpoint+margin:]
        left_ind = np.where(left_row == 1)
        right_ind = np.where(right_row == 1)
        if len(left_ind[0] != 0) and left_done == False:
            min_left_ind  = left_ind[0][0]
            max_left_ind  = left_ind[0][-1]
            med_left_ind  = left_ind[0][np.int32(len(left_ind[0])/2)]
            mean_left_ind = np.int32((min_left_ind+max_left_ind)/2)
            srcTop[0] = [max_left_ind,i]
            #srcTop[0] = [mean_left_ind,i]
            left_done = True
        if len(right_ind[0] != 0) and right_done == False:           
            min_right_ind = right_ind[0][0]
            max_right_ind = right_ind[0][-1]
            med_right_ind = right_ind[0][np.int32(len(right_ind[0])/2)]
            mean_right_ind = np.int32((min_right_ind+max_right_ind)/2)
            srcTop[1] = [row_midpoint+margin+min_right_ind,i]            
            #srcTop[1] = [row_midpoint+margin+mean_right_ind,i]            
            right_done = True
        if right_done == True and left_done == True:
            break
    
    # Extend source points to the top and bottom of the lane   
    if len(srcTop) != 0 and len(srcBottom) != 0:
        slopeLeft     = (srcBottom[0][1]-srcTop[0][1])/(srcBottom[0][0]-srcTop[0][0])
        slopeRight    = (srcBottom[1][1]-srcTop[1][1])/(srcBottom[1][0]-srcTop[1][0])
        interLeft     = -(slopeLeft*srcTop[0][0]-srcTop[0][1])
        interRight    = -(slopeRight*srcTop[1][0]-srcTop[1][1])
        ptBottomLeft  = [np.int32((image.shape[0]-interLeft)/slopeLeft),image.shape[0]]
        ptBottomRight = [np.int32((image.shape[0]-interRight)/slopeRight),image.shape[0]]
        ptTopLeft     = [np.int32((image.shape[0]/2+85-interLeft)/slopeLeft),np.int32(image.shape[0]/2+85)]
        ptTopRight    = [np.int32((image.shape[0]/2+85-interRight)/slopeRight),np.int32(image.shape[0]/2+85)]
        #if srcTop[0][1] <= srcTop[1][1]:
        #    ptTopLeft     = srcTop[0]
        #    ptTopRight    = [np.int32((srcTop[0][1]-interRight)/slopeRight),srcTop[0][1]]
        #else:
        #    ptTopLeft     = [np.int32((srcTop[1][1]-interLeft)/slopeLeft),srcTop[1][1]]
        #    ptTopRight    = srcTop[1]      
        src = np.int32([ptTopLeft,ptBottomLeft,ptBottomRight,ptTopRight])
        src = CheckSourcePoints(src,image)
    
    # It was observed that hard-coded numbers are tolerant to noise and work better
    src = [[580,450],[130,720],[1240,720],[710,450]]
    
    return src

## Check Source Points For Perspective Transform

In [ ]:
def CheckSourcePoints(src,image):
    global source
    s = image.shape
    if len(src) == 0 and len(source) != 0:
        src = source
    elif src[0][0] < 0 or src[1][0] < 0 or src[2][0] < 0 or src[3][0] < 0 or \
       src[0][0] > s[1] or src[1][0] > s[1] or src[2][0] > s[1] or src[3][0] > s[1] or \
       src[0][1] < 0 or src[1][1] < 0 or src[2][1] < 0 or src[3][1] < 0 or \
       src[0][1] > s[0] or src[1][1] > s[0] or src[2][1] > s[0] or src[3][1] > s[0]:
        src = source
    elif len(source) != 0 and (np.absolute((src[0][1]-src[1][1])) < 0.95*np.absolute((source[0][1]-source[1][1]))):
        src = source
    elif len(source) != 0 and (np.absolute((src[1][0]-src[2][0])) < 0.95*np.absolute((source[1][0]-source[2][0]))):
        src = source
    else:
        source = src
    return src

## Determine Destination Points For Perspective Transform

In [ ]:
def GetPerspectiveDestinationPoints(image,src):
    """
    This function estimates the destination points where the four corners in the original image 
    will map to in the warped image
    """ 
    #offset = 25
    #img_size = image.shape
    #dst = np.float32([[offset, offset], 
    #                  [offset, img_size[0]-offset], 
    #                  [img_size[1]-offset, img_size[0]-offset], 
    #                  [img_size[1]-offset, offset]])
    
    #img_size = image.shape
    #dst = np.int32([[240,0],[240,img_size[0]],[1040,img_size[0]],[1040,0]])
    
    img_size = image.shape
    offset_left  = src[1][0]
    offset_right = src[2][0] 
    dst = np.float32([[offset_left, 0], 
                      [offset_left, img_size[0]], 
                      [offset_right,img_size[0]], 
                      [offset_right,0]])

    
    return dst

## Horizontal Shift Function 

In [ ]:
def shiftHorizontally(image,pos):
    new_image = image
    for i in range(new_image.shape[0]):
        new_image[i] = shift(new_image[i],pos,mode='wrap')
    return new_image

## Image Processing Pipeline

In [ ]:
def ImageProcessPipeline(image):
    """
    This function contains the image processing pipeline of the video frames
    """ 
    # Undistort current video frame using the camera matrix (3D to 2D) and the distortion coefficients
    undistorted = UndistortImage(image, mtx, dist)
    
    # Use color transforms, gradients, etc., to create a thresholded binary image.
    color_thresh_hls = (180, 255)    
    color_thresh_luv = (225, 255)
    color_thresh_lab = (155, 200)
    grad_mag_thresh  = (85, 255)
    grad_dir_thresh  = (0.9, 1.1)

    color_binary_hls = ColorThresholdingHLS(undistorted, color_thresh_hls)
    color_binary_luv = ColorThresholdingLUV(undistorted, color_thresh_luv)
    color_binary_lab = ColorThresholdingLAB(undistorted, color_thresh_lab)
    grad_mag_binary  = GradientMagThreshold(undistorted, 3, grad_mag_thresh)
    grad_dir_binary  = GradientDirThreshold(undistorted, 3, grad_dir_thresh)
    
    ############# PLOT IMAGE ###########
    #plt.imshow(color_binary_hls,'gray')
    #plt.show()
    ####################################

    ############# PLOT IMAGE ###########
    #plt.imshow(color_binary_luv,'gray')
    #plt.show()
    ####################################

    ############# PLOT IMAGE ###########
    #plt.imshow(color_binary_lab,'gray')
    #plt.show()
    ####################################

    ############# PLOT IMAGE ###########
    #plt.imshow(grad_mag_binary,'gray')
    #plt.show()
    ####################################

    ############# PLOT IMAGE ###########
    #plt.imshow(grad_dir_binary,'gray')
    #plt.show()
    ####################################

    # Create composite binary image
    combined = np.zeros_like(color_binary_luv)
    combined[((color_binary_luv == 1) | \
              (color_binary_lab == 1) | \
              (grad_mag_binary == 1))] = 1

    
    ############# PLOT IMAGE ###########
    #plt.imshow(combined,'gray')
    #plt.show()
    ####################################
    
    
    ############# PLOT IMAGE ###########
    #imshape = image.shape
    #vertices = np.array([[(150,imshape[0]),
    #                      (imshape[1]/2.0-55, imshape[0]/2.0+75), 
    #                      (imshape[1]/2.0+55, imshape[0]/2.0+75), 
    #                      (imshape[1]-10,imshape[0])]], dtype=np.int32)
    #img = np.uint8(np.dstack((combined*255,combined*255,combined*255)))
    #proj0 = cv2.polylines(img,np.int32([vertices]),1,(0,0,255),5) 
    #plt.imshow(img)
    #plt.show()
    ####################################
     
    
    # Apply the mask to remove landscape features that might interfere with the CV analysis
    masked = ApplyMasking(combined)
    
    ############# PLOT IMAGE ###########
    #plt.imshow(masked,'gray')
    #plt.show()
    ####################################
     
    # Apply a perspective transform to rectify binary image ("birds-eye view").
    src  = GetPerspectiveSourcePoints(masked)
    dest = GetPerspectiveDestinationPoints(masked,src)
    
    #img = np.uint8(np.dstack((masked*255,masked*255,masked*255)))
    #proj1 = cv2.polylines(img,np.int32([src]),1,(255,0,0),5) 
    
    ############# PLOT IMAGE ###########
    #plt.imshow(proj1)
    #plt.show()
    ####################################
    
    # Given src and dst points, calculate the perspective transform matrix
    M = cv2.getPerspectiveTransform(np.float32([src]), np.float32([dest]))
    
    # Compute the inverse perspective transform:
    Minv = cv2.getPerspectiveTransform(np.float32([dest]), np.float32([src]))

    # Warp the image using OpenCV warpPerspective()
    warped = cv2.warpPerspective(masked, M, masked.shape[::-1])
    
    ############# PLOT IMAGE ###########
    #plt.imshow(warped,'gray')
    #plt.show()
    ####################################
    
    # Enhance the warped image knowing the lane lines are parallel
    #warped_mask = np.ones(warped.shape)
    #warped_mask[:,np.int32(warped_mask.shape[1]/2):] = 0
    #warped_masked = np.int32(warped) & np.int32(warped_mask)
    #warped_masked_shifted = np.array(warped_masked)
    #warped_masked_shifted = shiftHorizontally(warped_masked_shifted,800)
    #new_warped = warped_masked | warped_masked_shifted
    #warped = np.array(new_warped)
    
    ############# PLOT IMAGE ###########
    #plt.imshow(warped,'gray')
    #plt.show()
    ####################################

    #img = np.uint8(np.dstack((warped*255,warped*255,warped*255)))
    #proj2 = cv2.polylines(img,np.int32([dest]),1,(255,0,0),5) 
    
    ############# PLOT IMAGE ###########
    #plt.imshow(proj2)
    #plt.show()
    ####################################

    # Detect lane pixels and fit to find the lane boundary.
    leftBaseX, rightBaseX = FindLaneLineStart(warped)
    
    # Track lane lines with the method of sliding windows
    n_windows = 10
    left_lane_inds,right_lane_inds,nonzerox,nonzeroy,leftx,lefty,rightx,righty,out_img = \
    TrackSlidingWindows(n_windows,warped,leftBaseX,rightBaseX)
    
    # Fit a polynomial expression to the lane line points
    left_fit,right_fit,left_fitx,right_fitx,ploty = \
    FitPolynomialToSlidingWindows(warped,leftx,lefty,rightx,righty)

    ############# PLOT IMAGE ###########
    #plt.figure()
    #out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    #out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
    #plt.imshow(out_img)
    #plt.plot(left_fitx, ploty, color='yellow')
    #plt.plot(right_fitx, ploty, color='yellow')
    #plt.xlim(0, 1280)
    #plt.ylim(720, 0)
    #plt.show()
    ####################################

    # Visualize the lane lines
    #res1 = VisualizeLaneLines(warped,left_lane_inds,right_lane_inds,nonzerox,nonzeroy,left_fit,right_fit,100)
    
    ############# PLOT IMAGE ###########
    #plt.imshow(res1)
    #plt.show()
    ####################################
    
    # Warp the detected lane boundaries back onto the original image.
    res2 = DrawFullLane(undistorted,warped,Minv,left_fit,right_fit)

    # Determine the curvature of the lane and vehicle position with respect to center.
    left_curverad,right_curverad,leftx,rightx,ploty,left_fitx,right_fitx = ComputeLaneCurvature(warped,left_fit,right_fit)
        
    ############# PLOT IMAGE ###########
    #plt.figure()
    #mark_size = 3
    #plt.plot(leftx, ploty, 'o', color='red', markersize=mark_size)
    #plt.plot(rightx, ploty, 'o', color='blue', markersize=mark_size)
    #plt.xlim(0, 1280)
    #plt.ylim(0, 720)
    #plt.plot(left_fitx, ploty, color='green', linewidth=3)
    #plt.plot(right_fitx, ploty, color='green', linewidth=3)
    #plt.gca().invert_yaxis()
    #plt.grid()
    #plt.show()
    ####################################

    # Now our radius of curvature is in meters
    #print('Curvature: ',left_curverad, 'm,', right_curverad, 'm')
    
    # Compute car offset from the center of the lane
    offset = ComputeOffsetFromCenter(src,image)
    
    cv2.putText(res2, 'Radius of Curvature is ' + str((left_curverad+right_curverad)/2) + ' m', (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.25, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.putText(res2, 'Vehicle is ' + str(offset) + ' m left of center', (100, 150), cv2.FONT_HERSHEY_SIMPLEX, 1.25, (0, 255, 0), 2, cv2.LINE_AA)
    
    ############# PLOT IMAGE ###########
    #plt.imshow(res2)
    #plt.show()
    ####################################
    
    # Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.
    return res2

## Main Body

In [ ]:
# Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
mtx,dist,images = CalibrateCamera()

In [ ]:
# Apply a distortion correction to test images and showcase distortion correction performance.
#undistorted = UndistortImage(cv2.imread(images[0]), mtx, dist)
#plt.imshow(cv2.imread(images[0]))
#plt.show()
#plt.imshow(undistorted)
#plt.show()

# Make a list of calibration images
#test_images = glob.glob('./test_images/*.jpg')
    
# Step through the list, undistort and display images
#for fname in test_images:
#    print(fname)
#    imgBGR = cv2.imread(fname)
#    imgRGB = cv2.cvtColor(imgBGR,cv2.COLOR_BGR2RGB)
#    undistorted = UndistortImage(imgRGB, mtx, dist)
#    plt.imshow(imgRGB)
#    plt.show()
#    plt.imshow(undistorted)
#    plt.show()

In [ ]:
# Open the video clip and run the image processing pipeline on all image frames.
# Subsequently, save new video with the drawn full lane extent polygon. 

## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
clip = VideoFileClip("./project_video.mp4")
#clip = VideoFileClip("./project_video.mp4").subclip(1,3)
new_clip = clip.fl_image(ImageProcessPipeline) #NOTE: this function expects color images!!

In [ ]:
new_output = './project_video_output.mp4'
%time new_clip.write_videofile(new_output, audio=False)